In [2]:
# Imports & configuration

import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import altair as alt

alt.data_transformers.enable("vegafusion")


DataTransformerRegistry.enable('vegafusion')

# Ornstein-Uhlenbeck Process

The **Ornstein-Uhlenbeck process** is defined by the following stochastic differential equation:

$$dX_t = -\alpha X_t\,dt + \sigma dB_t $$

where $B_t$ is a standard Brownian motion.

It has the following solution:

$$
d(e^{\alpha t}X_t) = e^{\alpha t}\cdot\sigma\,dB_t\newline 
\Rightarrow X_t = e^{-\alpha t} X_0 + \int_0^t e^{\alpha(s-t)}\,dB_s
$$

In [3]:
# Define the SDE evolution
def ornstein_uhlenbeck(X0, alpha, sigma, dt):
    def next_value(dB, state={"X": X0}):
        X = state["X"]
        dX = -alpha * X * dt + sigma * dB

        state["X"] += dX
        return X
    
    return next_value

Suppose we want to hedge the claim $F=B_T$. We take the following position in $X_t$:

$$
\theta_t = \sigma^{-1}\left(1+\frac{\alpha}{\rho}\left(1-e^{\rho(t-T)}\right)\right)=\sigma^{-1}\left(1+\frac{\alpha}{\rho}-\frac{\alpha}{\rho}e^{\rho(t-T)}\right)
$$

Now let

$$
\begin{align*}
dA_t    &= \theta_t\,dX_t - d(\theta_t X_t)     \newline
        &= -\frac{\alpha}{\sigma\rho}e^{\rho(t-T)}\,dX_t + d\left(\frac{\alpha X_t}{\sigma\rho}e^{\rho(t-T)}\right)\newline
        &= \frac{\alpha}{\sigma}X_te^{\rho(t-T)}\,dt
\end{align*}
$$

Then the cash required to finance this position is

$$
\begin{align*}
\varphi_t       &= \varphi_0 + \int_0^t e^{-\rho s}\,dA_s\newline
                &= \varphi_0 + \int_0^t \frac{\alpha}{\sigma}X_se^{-\rho T}\,ds
\end{align*}
$$

Note that $X_s\,ds = \alpha^{-1}(\sigma\,dB_s - dX_s)$, so we get
$$
\begin{align*}
\varphi_t       &= \varphi_0 + \frac{\alpha}{\sigma}e^{-\rho T}\alpha^{-1}\left(\sigma B_t - X_t + X_0 \right)\newline
                &= \varphi_0 + e^{-\rho T}\left(B_t - \frac{X_t-X_0}{\sigma}\right)
\end{align*}
$$

Lastly we choose 
$$
\begin{align*}
\varphi_0       &= E_Q[e^{-\rho T}B_T]-\theta_0X_0\newline
                &= \frac{X_0}{\sigma}\left(1+\frac{\alpha}{\rho}\right)\left(1-e^{-\rho T}\right)-\frac{X_0}{\sigma}\left(1+\frac{\alpha}{\rho}\left(1-e^{-\rho T}\right)\right)\newline
                &= -\frac{X_0}{\sigma}e^{-\rho T}.
\end{align*}
$$

In [55]:
# Generate data

# Model parameters

T = 1
N = 10000
dt = T / N

# Risk-free rate
rho = 0.06

# Ornstein-Uhlenbeck process
X0 = 1.4
alpha = 1
sigma = 1


df = (
    pl.DataFrame()
    .with_columns(t=np.linspace(0, T, N), dB=np.random.normal(0, dt**0.5, size=N))
    .with_columns(
        M=(rho*pl.col("t")).exp(),
        X=pl.col("dB").map_elements(ornstein_uhlenbeck(X0, alpha, sigma, dt), pl.Float64),
        B=pl.cum_sum("dB"),
    )
    .with_columns(
        theta = (1 + alpha/rho*(1 - (rho*(pl.col("t") - T)).exp()))/sigma,
        cash = -X0 / sigma * pl.lit(-rho * T).exp() + pl.lit(-rho * T).exp() * (pl.col("B") - (pl.col("X") - X0)/sigma),
    )
    .with_columns(
        V = pl.col("theta") * pl.col("X") + pl.col("cash") * pl.col("M"),
    )
)

# Plot

alt.layer(
    # alt.Chart(df).mark_line(text='X').encode(x="t",y="X"),
    alt.Chart(df).mark_line(text='B', color="orange").encode(x="t", y="B"),
    alt.Chart(df).mark_line(text='V', color='green').encode(x="t", y="V")
).properties(width=800, height=600).interactive()
# df.plot.line(x="t", y="check")
# df.plot.line(x="t", y="B")

alt.LayerChart(...)